In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
sns.set() # use seaborn plotting style

: 

: 

Code from https://towardsdatascience.com/text-classification-using-naive-bayes-theory-a-working-example-2ef4b7eb7d5a

In [3]:
train_data = pd.read_csv('final_data/train.csv')
test_data = pd.read_csv('final_data/test.csv')

In [5]:
train_data.head()

,review,rating
0,"""This is the only medicine I have found that c...",1
1,"""Worked mildly well, but excellent when paired...",1
2,"""Insertion was not bad. For me it was fast and...",0
3,"""Works wonders. I have my life back. The first...",1
4,"""I have just finished my second week on Tri-Sp...",0


In [6]:
# Build the model
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
# Train the model using the training data
model.fit(train_data.review, train_data.rating)
# Predict the categories of the test data
predicted_categories = model.predict(test_data.review)

In [8]:
print(np.array(test_data.rating)[predicted_categories])

[1 1 1 ... 1 1 1]


In [10]:
# plot the confusion matrix
mat = confusion_matrix(test_data.rating, predicted_categories)
sns.heatmap(mat.T, square = True, annot=True, fmt = "d", xticklabels=train_data.rating,yticklabels=train_data.rating)
plt.xlabel("true labels")
plt.ylabel("predicted label")
plt.show()

KeyboardInterrupt: 

In [ ]:
print("The accuracy is {}".format(accuracy_score(test_data.target, predicted_categories)))